# SNLP Assignment 3

Name 1: Parnian Jahangirirad    
Student id 1: 7062810  
Email 1: paja00003@stud.uni-saarland.de    



**Instructions:** Read each question carefully. <br/>
Make sure you appropriately comment your code wherever required. Your final submission should contain the completed Notebook and the respective Python files for any additional exercises necessary. There is no need to submit the data files should they exist. <br>
**In this case, however, submit your randomized text for `randomization_parameter` = 0.2 from exercise 2.2, so that we can verify your code.**  <br/>

Upload the zipped folder on CMS. Please follow the naming convention of **Name1_studentID1_Name2_studentID2_Name3_studentID3.zip**. Only one member of the group should make the submisssion.

---

### General guidelines:
1. Don't change the layout of the notebook, especially the final output cells. If you do end up changing the code, add comments to tell us why you needed to do it.
2. Use log with base 2 wherever you feel logarithimic operations are necessary. If you do use other bases, please specify it and explain why.
3. For tokenizers, it is sufficient to use tokenizers from `nltk` library.
4. Always use log with base 2 for your logarithmic operations. 
5. For the theoritical questions where no math is involved, it is sufficient to answer the questions with 3-4 sentences. Do not be too verbose.

---

# Exercise 1 - Joint and conditional entropy (1 point)

In this exercise you will review and work on information theory concepts.

We define the joint entropy as:

$$ H(X,Y) = - \sum_{x \in \mathcal{X}} \sum_{y \in \mathcal{Y}} p(x,y) \log p(x,y) $$

We define the conditional entropy as:

$$
\begin{align}
H(Y|X) & = \sum_{x \in \mathcal{X}} p(x,y) H(Y|X=x) \\
& = \sum_{x \in \mathcal{X}} p(x) \left[ - \sum_{y \in \mathcal{Y}}p(y|x) \log p(y|x) \right]
\end{align}
$$

Show that $H(X,Y) = H(X) + H(Y|X)$ or that $ H(Y|X) = H(X,Y) - H(X) $ holds.

![Exersice1](exersice1.jpg)

# Exercise 2: Perplexity and Entropy

In this exercise, you will explore the concept of entropy and perplexity in the context of text analysis. You will calculate these measures for a given text corpus, investigate how they change when the text is altered, and analyze the impact of combining texts from different languages.

### Exercise 2.1: Perplexity and Entropy calculation (2 points)

You're given the text of Alice in Wonderland: `alice_eng.txt`. Now using the text file, perform the following steps.

1. Load the text and tokenize and lowercase each token. Filter out the puncatuation also. (0.5 points)

2. Compute $P(i,j)$, which is the probability that at any position in the text you will find the word $i$ followed immediately by the word j, and $P(j|i)$, which is the probability that if word $i$ occurs in the text then word $j$ will follow. (0.5 points)

3. Calculate the the conditional entropy of the word distribution which can also be written as: (0.5 point) $$H(J|I) = -\sum_{i \in I, j \in J} P(i,j)log_{2}P(j|i)$$

4. And then, the perplexity: (0.5 points)$$PP = 2^{H(J|I)}$$

To accomplish all the above, complete the functions `compute_probabilities()`, `compute_entropy()` and `compute_perplexity()`, then run the driver code below

In [1]:
import nltk
nltk.download('punkt')
from collections import Counter
import math
import random
import numpy as np
from nltk import FreqDist
from math import log2
import string

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pjrad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# We can define Preprocessing function in order to satisfy part1 of Q2.1
from nltk.tokenize import RegexpTokenizer
def preprocessing(filepath):
    #load the text
    with open(filepath, 'r', encoding='utf_8') as file:
        text = file.read()

    #lowercasing
    text = text.lower()

    #tokenizing and removing punctuations
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    #print("type of tokens is: ",type(tokens))
    return tokens

In [3]:
from nltk import bigrams
def compute_probabilities(filename):
    """
    Compute the probabilities P(i, j) and P(j | i) for a given text.

    Parameters:
    filename (str): The name of the text file.

    Returns:
    p_ij (dict): A dictionary where the keys are bigrams (i, j) and the values are the probabilities P(i, j).
    p_j_given_i (dict): A dictionary where the keys are bigrams (i, j) and the values are the conditional probabilities P(j | i).
    """
    #getting preprocessed tokens
    tokens = preprocessing(filename)

    #calculating bigrams and their counts
    bigram_freq = FreqDist(bigrams(tokens))
    unigram_freq = FreqDist(tokens)

    #Calculating P(i, j) and P(j | i)
    total_bigrams = sum(bigram_freq.values())

    p_ij = {}
    p_j_given_i = {}
    
    for bigram, count in bigram_freq.items():
        word1, word2 = bigram
        p_ij[bigram] = count / total_bigrams
        p_j_given_i[bigram] = count / unigram_freq[word1]
    
    return p_ij, p_j_given_i


    pass

In [4]:
def compute_entropy(p_ij, p_j_given_i):
    """
    Compute the conditional entropy of the word distribution in a text given the previous word.

    Parameters:
    p_ij (dict): A dictionary where the keys are bigrams (i, j) and the values are the probabilities P(i, j).
    p_j_given_i (dict): A dictionary where the keys are bigrams (i, j) and the values are the conditional probabilities P(j | i).

    Returns:
    float: The conditional entropy of the word distribution in the text given the previous word.
    """
    #initialize the conditional entropy
    H_j_given_i = 0

    for bigram in p_ij:
        i, j = bigram
        p_ij_value = p_ij[bigram]
        p_j_given_i_value = p_j_given_i[bigram]
        if p_ij_value > 0 and p_j_given_i_value > 0:
            H_j_given_i -= p_ij_value * math.log2(p_j_given_i_value)


    return H_j_given_i
    pass


def compute_perplexity(entropy):
    """
    Compute the perplexity of a text given its entropy.

    Parameters:
    entropy (float): The entropy of the text.

    Returns:
    float: The perplexity of the text.
    """

    return 2**entropy
    pass

In [5]:
p_ij, p_j_given_i = compute_probabilities('data/alice_eng.txt')
entropy = compute_entropy(p_ij, p_j_given_i)
perplexity = compute_perplexity(entropy)

print(f'Entropy: {entropy}, Perplexity: {perplexity}')

Entropy: 3.6853507360231506, Perplexity: 12.864743041112018


### Exercise 2.2: Altering the text (2 points)

Now, take the text, randomize its contents and perform the following operations:

1. The `randomization` parameter determines the fraction of words that will be replaced. If parameter is $n$, randomly select $n$% of words from the text and replace it with a randomly chosen word from the set of words that appear in the text. (0.5 points)

2. Since there is some randomness to the outcome of the experiment, run the experiment 5 times. From all 5 trials, calculate the minimum, maximum, and average entropy from the experiments. Also calculate average perplexity.(0.5 points)


Do this experiment with different `randomization` values: 1%, 5%, 10%, 20%.

To accomplish all the above, complete the functions `randomize_text()`, `run_experiment()`, then run the driver code below

**Note**: 
1. Save the randomized version of the text for the `randomization` parameter: 0.2, you will need it for Exercise 3.
2. Use the supplied seed parameter `seed_val`, don't change the value.

In [6]:
def randomize_text(filename, randomization):
    """
    Randomize a fraction of words in a text.

    Parameters:
    filename (str): The name of the text file.
    randomization (float): The fraction of total words that will be replaced with a random word.

    Returns:
    str: The randomized text.
    """
    rtokens = preprocessing(filename)

    # number of tokens to be replaced
    total_tokens = len(rtokens)
    random_num = int(total_tokens * randomization)

    unique_tokens = list(set(rtokens))

    # Randomly select tokens to be replaced
    indices_to_replace = random.sample(range(total_tokens), random_num)
    for index in indices_to_replace:
        rtokens[index] = random.choice(unique_tokens)


    return rtokens


    pass

def run_experiment(filename, randomization, seed_val=0):
    """
    Run an experiment to randomize a text and compute the entropy of the resulting text.

    Parameters:
    filename (str): The name of the text file.
    randomization (float): The fraction of total words that will be replaced with a random word.

    Returns:
    tuple: A tuple containing the minimum, maximum, and average entropy of the randomized text over 5 runs of the experiment.
    """
    generated_text = ''
    random.seed(seed_val)
    entropies = []
    #running experiments 5 times
    for i in range(5):
        #generating randomized tokens
        tokens = randomize_text(filename, randomization)
        generated_text = ' '.join(tokens)
        #print(type(tokens))
        #calculating probabilities
        #calculating bigrams and their counts
        bigram_freq = FreqDist(bigrams(tokens))
        unigram_freq = FreqDist(tokens)

        #Calculating P(i, j) and P(j | i)
        total_bigrams = sum(bigram_freq.values())

        p_ij = {}
        p_j_given_i = {}
            
        for bigram, count in bigram_freq.items():
            word1, word2 = bigram
            p_ij[bigram] = count / total_bigrams
            p_j_given_i[bigram] = count / unigram_freq[word1]

        #computing entropy
        entropies.append(compute_entropy(p_ij, p_j_given_i))
##################
    min_entropy = min(entropies)
    max_entropy = max(entropies)
    avg_entropy = sum(entropies) / len(entropies)


    #save the text with randomization parameter= 0.2
    if randomization == 0.2:
        # Save the generated text to a file
        with open('randomized_0.2.txt', 'w', encoding='utf-8') as file:
            file.write(generated_text)

    
    return (min_entropy, max_entropy, avg_entropy)
    pass

In [7]:
# Run the experiment for 5 iterations
randomization_params = [0.01, 0.05, 0.1, 0.2]
for randomization in randomization_params:
  min_entropy, max_entropy, avg_entropy = run_experiment('data/alice_eng.txt', randomization)
  print(f'Min Entropy: {min_entropy}, Max Entropy: {max_entropy}, Average Entropy: {avg_entropy}, Average perplexity: {compute_perplexity(avg_entropy)} at randomization parameter = {randomization*100}%')


Min Entropy: 3.687757796849216, Max Entropy: 3.690913188542005, Average Entropy: 3.6892492118415534, Average perplexity: 12.899553386554498 at randomization parameter = 1.0%
Min Entropy: 3.699309624282209, Max Entropy: 3.701776876653441, Average Entropy: 3.700092847713372, Average perplexity: 12.996874756352831 at randomization parameter = 5.0%


Min Entropy: 3.6967393275950515, Max Entropy: 3.7059688847122825, Average Entropy: 3.7000044962112155, Average perplexity: 12.996078844385885 at randomization parameter = 10.0%
Min Entropy: 3.6575044052364, Max Entropy: 3.685702866039686, Average Entropy: 3.667494567775607, Average perplexity: 12.706498040848436 at randomization parameter = 20.0%


After running the above experiments, answer the below questions: (1 point)

1. How does altering a text (like randomizing or messing up words and characters) affect its entropy and perplexity in your opinion? Does the results from the experiment match with your expectations? 

  A: In my opinion, it should increase entropy and perplexity. However, the result do not show a noticable change in average entropy or perplexity.  

2. How would the entropy and perplexity change if we were to consider trigrams (sequences of three words) instead of bigrams?

  A: I think that altering text would have a larger impact on entropy and perplexity if we were using trigrams instead of bigrams, since trigram uses a longer sequence of tokens, so it will be more sensible to changes we make in text.  

### Exercise 2.3 : Character level analysis (2 points)

Perform the same operation as the previous exercise, but this time with characters instead of words, i.e. consider the `randomization_parameter` to be the fraction of characters, not words to be replaced randomly. (1 point)

In [8]:
import string
def randomize_char(filename, likelihood):
    """
    Randomize a fraction of characters in a text.

    Parameters:
    filename (str): The name of the text file.
    likelihood (float): The fraction of total characters that will be replaced with a random character.

    Returns:
    str: The randomized text.
    """
    #loading the file
    with open(filename,'r',encoding='utf_8') as file:
        text = file.read()

    #lowercasing
    text = text.lower()

    #removing punctuations
    text.translate(str.maketrans('', '', string.punctuation))

    #tokenizing in character level
    ch_tokens = list(text)

    # number of tokens to be replaced
    total_tokens = len(ch_tokens)
    random_num = int(total_tokens * randomization)

    unique_tokens = list(set(ch_tokens))

    # Randomly select tokens to be replaced
    indices_to_replace = random.sample(range(total_tokens), random_num)
    for index in indices_to_replace:
        ch_tokens[index] = random.choice(unique_tokens)

    return ch_tokens
    pass

def run_experiment_char(filename, randomization, seed_val=42):
    """
    Run an experiment to randomize a text and compute the entropy of the resulting text.

    Parameters:
    filename (str): The name of the text file.
    likelihood (float): The fraction of total characters that will be replaced with a random character.

    Returns:
    tuple: A tuple containing the minimum, maximum, and average entropy of the randomized text over 5 runs of the experiment.
    """

    tokens = randomize_char(filename, randomization)

    random.seed(seed_val)
    entropies = []
    #running experiments 5 times
    for i in range(5):
        #generating randomized tokens
        tokens = randomize_text(filename, randomization)
        #print(type(tokens))
        #calculating probabilities
        #calculating bigrams and their counts
        bigram_freq = FreqDist(bigrams(tokens))
        unigram_freq = FreqDist(tokens)

        #Calculating P(i, j) and P(j | i)
        total_bigrams = sum(bigram_freq.values())

        p_ij = {}
        p_j_given_i = {}
            
        for bigram, count in bigram_freq.items():
            word1, word2 = bigram
            p_ij[bigram] = count / total_bigrams
            p_j_given_i[bigram] = count / unigram_freq[word1]

        #computing entropy
        entropies.append(compute_entropy(p_ij, p_j_given_i))

    min_entropy = min(entropies)
    max_entropy = max(entropies)
    avg_entropy = sum(entropies) / len(entropies)
    
    return (min_entropy, max_entropy, avg_entropy)
        
    pass

In [9]:
# Run the experiment for 5 iterations
randomization_params = [0.01, 0.05, 0.1, 0.2]
for randomization in randomization_params:
  min_entropy, max_entropy, avg_entropy = run_experiment_char('data/alice_eng.txt', randomization)
  print(f'Min Entropy: {min_entropy}, Max Entropy: {max_entropy}, Average Entropy: {avg_entropy}, Average perplexity: {compute_perplexity(avg_entropy)} at randomization parameter = {randomization*100}%')

Min Entropy: 3.684199983225344, Max Entropy: 3.693319952014129, Average Entropy: 3.6895256557501668, Average perplexity: 12.902025388280757 at randomization parameter = 1.0%
Min Entropy: 3.6855632137898304, Max Entropy: 3.714533983052362, Average Entropy: 3.6999798550133294, Average perplexity: 12.99585687355584 at randomization parameter = 5.0%
Min Entropy: 3.6909175918304142, Max Entropy: 3.7160088867975025, Average Entropy: 3.705847604572029, Average perplexity: 13.048821441495809 at randomization parameter = 10.0%
Min Entropy: 3.6667280591983618, Max Entropy: 3.677514269500429, Average Entropy: 3.672032475076642, Average perplexity: 12.746528461795275 at randomization parameter = 20.0%


After running the above experiments, answer the below questions: (1 point)

1. What changes do you observe in the entropy values? Why do you think the values are different from the ones at the word-level? (0.5 points)

  A: There are more noticable increases in the entropy and perplexity (but not as much as I expected). I expected the values to be way more different from what we had at word-level, since in character_level tokens, the next token should be less predictable.  

2. If the same operation was performed on a character-level in another language, (for instance German), do you think the entropy will be more or less? (0.5 points)

  A: If the language has more characters than what English, then the entropy would be higher.  
  (If we're talking about an English text which its characters are randomly replaced by characters of another language, then the entropy definitely increases).  

# Exercise 3:  KL-Divergence (3 points)

Kl-Divergence is a measure of how dissimilar two different probability distributions are from each other. It is used in language modelling by comapring different distributions with each other. It is defined as:

\begin{equation}
D_{KL}(P\|Q) = \sum_{x \in X}P(x) \cdot \log \frac{P(x)}{Q(x)}
\end{equation}

Where $P$ is the empirical or observed distribution, and Q is the estimated distribution over a common probabilitiy space $X$.

In this exercise, you will be comparing the probability distrubutions of Alice in Wonderland: `alice_eng.txt` with it's German edition (`alice_ger.txt`) as well as the 20% randomized version of the English edition. To do that, follow the steps:

1. Read the text files and tokenize the text into individual words and filter out punctuation. (0.5 points)

2. Compute the frequency distribution of the unigrams from their respective texts. (0.5 points)

3. Compute the probabilities of each word (unigram) in their respective texts using the frequency distribution. (0.5 points)

4. Use these probabilities to calculate the KL Divegence. If a word from the first distribution is not available in the 2nd distribution, then do not include it in the divergence calculation. (0.5 points)


In [10]:
def calculate_kl_divergence(file1, file2):
    """
    Calculate the Kullback-Leibler (KL) Divergence between two text files.

    This function reads two text files, tokenizes the text into individual words, calculates the frequency distribution of words in each text, computes the probabilities of each word in each text, and finally calculates the KL divergence.

    Parameters:
    file1 (str): The name of the first text file.
    file2 (str): The name of the second text file.

    Returns:
    kl_divergence (float): The KL divergence between the two texts. For each word in the first text, if the word also exists in the second text, the product of the probability of the word in the first text and the logarithm base 2 of the ratio of the probability of the word in the first text to the probability of the word in the second text is added to the KL divergence.
    """
    # Read the files
    # Remove punctuation
    # Tokenize the text
    # Get the frequency distribution
    # Calculate the probabilities
    # Calculate the KL divergence

    
    #preprocessing
    tokens1 = preprocessing(file1)
    tokens2 = preprocessing(file2)

    #compute frequency distrubutions
    distr_freq1 = FreqDist(tokens1)
    distr_freq2 = FreqDist(tokens2)

    #computing probabilities
    total_count1 = sum(distr_freq1.values())
    distr_prob1 = {word: count / total_count1 for word, count in distr_freq1.items()}

    total_count2 = sum(distr_freq2.values())
    distr_prob2 = {word: count / total_count2 for word, count in distr_freq2.items()}

    # Calculate KL divergence
    kl_divergence = 0.0
    for word, p_x in distr_prob1.items():
        if word in distr_prob2:
            q_x = distr_prob2[word]
            kl_divergence += p_x * math.log2(p_x / q_x)
    
    return kl_divergence
    pass

In [11]:
kl_divergence_eng_de = calculate_kl_divergence('data/alice_eng.txt', 'data/alice_ger.txt')
kl_divergence_eng_random = calculate_kl_divergence('data/alice_eng.txt', 'randomized_0.2.txt')
print(f'The KL-divergence between the text and its translation is: {kl_divergence_eng_de}')
print(f'The KL-divergence between the text and its 20% randomized version is: {kl_divergence_eng_random}')

The KL-divergence between the text and its translation is: 1.6498665567036532
The KL-divergence between the text and its 20% randomized version is: 0.09616660765861819


After running the above experiments, answer the below questions: (1 point)

1. What do you think of the divergence values you have calculated? Do they match with your expectations?

  A: The KL-divergence between the text and its 20% randomized version is lower than the KL-divergence value between the text and its translation. This result could be expected because different languages have different vocabularies, which increases the dissimilarity between the two texts compared to when we only change 20% of text's words with words from the same language.  

2. Do you think this is a good way to compare different versions of a text?

  A: KL-divergence is a good way to compare different versions of a text based on their probabilistic distributions. However, there are some weaknesses, which the most important one in my opinion is the lack of symmetry, meaning 
  $D_{KL}(P\|Q) \neq D_{KL}(Q\|P)$. A good measurement should be able to satisfy symmetry.  
  In addition, KL-divergence just considers the probability distributions, and doesn't consider the context of the text. Two texts with similar word distributions but different meanings can have a low KL-divergence.  

# Exercise 4 (Bonus): Language Models and Compression (3 points)

This is a reading asignment, read the following two papers and answer the below questions.

Paper 1: [Language Modelling is Compression](https://arxiv.org/pdf/2309.10668)<br>
Paper 2: [Low-Resource Text Classification](https://aclanthology.org/2023.findings-acl.426.pdf)

Q1. Paper 1 draws relationships between probabilistic ML models and lossless compression techniques. Why not the same parallels with lossy compression methods?

  A: Lossless compression allows perfect reconstruction of the original data, while lossy compression allows some information loss for more improved compression rates. Lossy compression methods are generally designed to reduce the amount of data by approximating the original content. This process is often done by removing some of the less important information, it involves an inherent exchange between compression ratio and fidelity to the original data. On the other hand, Probabilistic models and lossless compression techniques, focus on predicting and encoding data accurately without any loss of information. The foundational theories and algorithms in probabilistic modeling and lossless compression, such as Shannon's source coding theorem, are based on preserving the entire information content, which aligns nicely with the goals of predictive modeling. Therefore, the similarities are more straightforward and based on the theory of lossless compression, and cannot be achieved with lossy compression methods.  

Q2. Looking over the results of Paper 1, do you think there is any advantage is using LLMs as compressors? What could be a possible roadblock?

  A: Firstly, we look at the meaning of the LLMs and compressors separately. Large Language Models are processing massive amounts of text data to be capable of understanding and generating human language. The concept of "LLMs as compressors" refers to the idea of using large language models (LLMs) as a means of compressing information. Compression generally is the process of encoding information using fewer bits than the original form of it. Compression in language models involves leveraging the models' understanding and ability to generate text to represent large amounts of information more concisely.
  
  Advantages:
  
  LLMs can compress data across multiple modalities (For example: text, images, and audio) more effectively and efficiently, surpassing various domain-specific compressors.
  
  LLMs' in-context learning capabilities can be used for efficient offline compression by keeping the model parameters fixed and only encoding the input data.
  
  Looking at how LLMs perform when compressed can give us new ideas about how to make these models work better and faster.
  
  
  Possible roadblock:
  The major limitation is that the large model size of LLMs needs to be encoded as part of the compressed file size. When working with small datasets, the extra work cancels any space-saving benefits gained from using powerful models. As a result, LLM compression may only have the advantage for extremely large datasets. Finding ways to reduce the model size overhead could make LLM compression more practical.  

Q3. In Paper 2, why do you think the gzip-based algorithm performs so well on OOD datasets when compared to more traditional approaches?

 A:  The key reason why the gzip-based algorithm performs so well on out-of-distribution (OOD) datasets compared to traditional approaches like deep neural networks is because it does not rely on any pre-training or make assumptions about the data distribution. Traditional deep learning models like BERT are pre-trained on large corpora of text data. While this allows them to achieve state-of-the-art performance on in-distribution data that matches their pre-training data, their performance can degrade significantly on OOD data from different distributions that they were not pre-trained on, like low-resource languages.

In contrast, the gzip algorithm is completely non-parametric- it does not have any trainable parameters and does not make assumptions about the data. It relies solely on the idea that similar texts will compress to similar lengths when concatenated. This compression property seems to transfer better to OOD data distributions compared to the learned representations in deep learning models.
 
 In case of the reasons why the gzip-based algorithm performs so well on OOD datasets, the paper suggests a few reasons:
 
    Compressors like gzip are able to capture underlying regularities in text that generalize well, even for languages and domains not seen during training.

    The compressor-based distance metric (NCD) is effective at quantifying semantic similarity, which is key for handling OOD scenarios.

    The kNN classifier is able to leverage this compressed representation to make accurate predictions, without requiring extensive fine-tuning or adaptation.

In contrast, traditional deep learning approaches may struggle with OOD data due to overfitting to the training distribution and the need for large amounts of labeled data for fine-tuning. The simplicity and universality of the gzip-kNN approach appears to provide better generalization to unseen settings.  

  
  References:
    OpenAI. (2024). ChatGPT (3.5) [Large language model]